# **ETL - Extracción, transformación y carga**

### IMPORTACIÓN DE LIBRERÍAS 

In [1]:
import re
import pandas as pd 
import numpy as np 
from summarytools import dfSummary
import matplotlib.pyplot as plt
import seaborn as sns 
from pandasgui import show
import plotly.express as px
import calendar

## Exploración y lecturas de los DataSets

In [2]:
df_siniestros = pd.read_excel('C:\\Users\\ACER\\Documents\\HENRY\\LABS\\PI02\\DATA\\homicidios.xlsx', sheet_name="HECHOS")
df_victimas = pd.read_excel('C:\\Users\\ACER\\Documents\\HENRY\\LABS\\PI02\\DATA\\homicidios.xlsx', sheet_name="VICTIMAS")
Barrios = pd.read_excel("C:\\Users\\ACER\\Documents\\HENRY\\LABS\\PI02\\Barrios por comuna.xlsx") 
#Se adjunta un dataset externo con el fin de completar nuestro tratamiento de datos al momento de hacer la transformación. 

In [3]:
df_siniestros.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


## Tratamiento y transformación de datos DF_SINIESTROS

In [4]:
df_siniestros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [5]:
df_siniestros.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

In [6]:
num_filas = df_siniestros.shape[0]
num_columnas = df_siniestros.shape[1]
print(f'Número de filas: {num_filas}')
print(f'Número de columnas: {num_columnas}')


Número de filas: 696
Número de columnas: 21


Verificación de datos nulos

In [7]:
df_siniestros.isnull().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

Se visualizan los datos nulos

In [8]:
df_siniestros[["Altura", "Calle", "Dirección Normalizada", "Cruce"]][:3]

,Altura,Calle,Dirección Normalizada,Cruce
0,NaN,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...","FERNANDEZ DE LA CRUZ, F., GRAL. AV."
1,NaN,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",DE LOS CORRALES AV.
2,2034.0,ENTRE RIOS AV.,ENTRE RIOS AV. 2034,NaN


In [9]:
# Filtrar filas con dirección normalizada nula y seleccionar columnas para poder realizar una visualización completa
subset_df = df_siniestros.loc[df_siniestros["Dirección Normalizada"].isnull(), ["Dirección Normalizada", "Cruce", "Altura", "Calle"]]
subset_df

,Dirección Normalizada,Cruce,Altura,Calle
38,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
106,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA
119,NaN,NaN,NaN,NaN
180,NaN,NaN,NaN,AUTOPISTA PERITO MORENO
181,NaN,NaN,NaN,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.
313,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
546,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
621,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA


Después de analizar los datos faltantes, que representan el 5,1% del conjunto de datos y están distribuidos en tres columnas: Altura, Dirección Normalizada, Cruce y Calle, se observaron las siguientes situaciones:

- Para la columna "Calle", se encontró un único valor nulo.
- La columna "Dirección Normalizada" tenía 8 valores nulos.
- En cuanto a la columna "Cruce", se identificaron 171 valores nulos, lo que equivale al 24,56% de los datos en esa columna.
- En la columna "Altura", se encontraron 567 valores nulos, lo que representa el 81,5% de los datos de esa columna.

Tras analizar los datos de la columna "Altura", se decidió eliminarla debido a la gran cantidad de valores faltantes y la imposibilidad de llenarlos. La columna "Altura" hace referencia a la numeración de la calle donde ocurre el incidente, y la mayoría de los casos ocurren en cruces o esquinas, lo que la hace poco útil.

La columna "Cruce" solo tendrá datos cuando el incidente ocurra en el cruce de calles, por lo que se conservará de esta manera. 

Por otro lado, la columna "Dirección Normalizada" presenta solo un 1,1% de valores nulos, por lo que se conservará sin realizar cambios, ya que no hay posibilidad de completarla con información de otras columnas del conjunto de datos.

In [10]:
df_siniestros = df_siniestros.drop('Altura', axis= 1)
df_siniestros.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

Se verifica la corrección realizada

In [11]:
df_siniestros.duplicated().sum()

0

In [12]:
df_siniestros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Cruce                  525 non-null    object        
 12  Dirección Normalizada  688 non-null    object        
 13  COMUN

In [13]:
df_siniestros.head(5)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


Al examinar el conjunto de datos, se hace evidente que la primera columna, 'HH', requiere una transformación en el tipo de datos. Actualmente, se encuentra almacenada como un objeto, pero debería ser de tipo numérico. Por lo tanto, se procede a realizar la conversión necesaria a un tipo de dato numérico.

In [14]:
df_siniestros["HH"] = pd.to_numeric(df_siniestros["HH"], errors="coerce", downcast="integer")
df_siniestros.info() # Se verifica la corrección 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     695 non-null    float64       
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Cruce                  525 non-null    object        
 12  Dirección Normalizada  688 non-null    object        
 13  COMUN

In [15]:
df_siniestros.drop('HORA', axis=1, inplace=True)
#y miro HH
df_siniestros['HH'].apply(type).value_counts()

HH
<class 'float'>    696
Name: count, dtype: int64

In [16]:
df_siniestros_hh = df_siniestros['HH'].mode().iloc[0]
# Reemplaza las filas donde 'HH' es de tipo str con la moda calculada
df_siniestros.loc[df_siniestros['HH'].apply(type) == str,'HH']= df_siniestros_hh

Los nombres de las columnas están en formato mixto, con algunas en mayúsculas y otras en minúsculas. 
Para seguir buenas prácticas, se normalizan a un formato consistente.

In [17]:
# Cambiar la primera letra a mayúscula y reemplazar guiones por espacios
df_siniestros.columns = df_siniestros.columns.str.capitalize().str.replace("_", " ")

# Renombrar columnas específicas
df_siniestros.rename(columns={"N victimas": "Num victimas",
                              "Aaaa": "Año",
                              "Mm": "Mes",
                              "Dd": "Día",
                              "Hh": "Hora int"}, inplace=True)
#Se procede a visualizar el cambio realizado 
df_siniestros.columns 

Index(['Id', 'Num victimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora int',
       'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado'],
      dtype='object')

Vamos a extraer las coordenadas de la columna **"XY (CABA)"** ya que planeamos utilizarlas posteriormente para representar los datos en un mapa. 

In [18]:
# Función para extraer las coordenadas de un texto
def extraer_coordenadas(texto):
    coordenadas = re.findall(r'\d+\.\d+', texto)
    return tuple(map(float, coordenadas)) if len(coordenadas) == 2 else (None, None)

# Aplicar la función y filtrar registros
df_siniestros[['coordenada_x', 'coordenada_y']] = df_siniestros['Xy (caba)'].apply(extraer_coordenadas).apply(pd.Series)
df_siniestros.dropna(subset=['coordenada_x', 'coordenada_y'], inplace=True)
df_siniestros.head(5)


,Id,Num victimas,Fecha,Año,Mes,Día,Hora int,Lugar del hecho,Tipo de calle,Calle,...,Dirección normalizada,Comuna,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,coordenada_x,coordenada_y
0,2016-0001,1,2016-01-01,2016,1,1,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,...,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378
1,2016-0002,1,2016-01-02,2016,1,2,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",...,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420
2,2016-0003,1,2016-01-03,2016,1,3,7.0,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,...,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878
3,2016-0004,1,2016-01-10,2016,1,10,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,...,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD,99840.652248,94269.165344
4,2016-0005,1,2016-01-21,2016,1,21,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,...,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS,106980.328279,100752.169158


Al examinar los datos en la columna 'Día', se observó que solo contenían números que representaban el día del mes, sin información sobre el nombre del día de la semana al que correspondía. Para facilitar un análisis más detallado y comprender mejor cómo se relacionan los siniestros con los diferentes días de la semana, se procedió a crear una nueva columna llamada 'Día de la Semana'. En esta nueva columna, cada fecha se asignó al día de la semana correspondiente, permitiendo así un análisis más completo de los patrones de incidentes en función de los días específicos de la semana.

In [19]:
# Se una nueva columna 'Día de la Semana' utilizando la función map y el módulo calendar
df_siniestros['Día de la Semana'] = df_siniestros['Fecha'].dt.dayofweek.map(
    {0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves', 4: 'Viernes', 5: 'Sábado', 6: 'Domingo'}
)
df_siniestros.head(3)

,Id,Num victimas,Fecha,Año,Mes,Día,Hora int,Lugar del hecho,Tipo de calle,Calle,...,Comuna,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,coordenada_x,coordenada_y,Día de la Semana
0,2016-0001,1,2016-01-01,2016,1,1,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,...,8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378,Viernes
1,2016-0002,1,2016-01-02,2016,1,2,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",...,9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420,Sábado
2,2016-0003,1,2016-01-03,2016,1,3,7.0,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,...,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878,Domingo


En el contexto de mi proyecto de transformación de datos, utilicé información relevante disponible en la página web oficial del Gobierno de la Ciudad de Buenos Aires (https://buenosaires.gob.ar/comunas). 
Esta fuente de datos proporciona detalles importantes sobre las divisiones comunales de la ciudad y los barrios que las componen. La información recopilada de esta fuente se utilizó con el fin de enriquecer y mejorar el conjunto de datos como lo mostraré a continuación al utilizar el dataset 'Barrios'

In [20]:
# Crear un diccionario de mapeo entre 'Comuna' y 'Barrios' de comunas
mapeo_barrios = Barrios.set_index('Comuna')['Barrios'].to_dict()

# Aplicar el mapeo utilizando la columna 'Comuna' en 'homicidios_hechos'
df_siniestros['Barrios'] = df_siniestros['Comuna'].map(mapeo_barrios)


In [21]:
df_siniestros.head(1)

,Id,Num victimas,Fecha,Año,Mes,Día,Hora int,Lugar del hecho,Tipo de calle,Calle,...,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,coordenada_x,coordenada_y,Día de la Semana,Barrios
0,2016-0001,1,2016-01-01,2016,1,1,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,...,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378,Viernes,"Villa Soldati, Villa Riachuelo y Villa Lugano"


Se procede a eliminar la columna **'XY (CABA)'** ya que contamos con los datos desglozados 

In [22]:
df_siniestros = df_siniestros.drop('Xy (caba)', axis= 1)
df_siniestros.columns

Index(['Id', 'Num victimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora int',
       'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Pos x', 'Pos y', 'Participantes',
       'Victima', 'Acusado', 'coordenada_x', 'coordenada_y',
       'Día de la Semana', 'Barrios'],
      dtype='object')

El siguiente código `dfSummary(df_siniestros)` se utiliza para resumir y comprender rápidamente el conjunto de datos en nuestro DataFrame `df_siniestros`. Proporciona estadísticas descriptivas, información sobre valores faltantes, distribuciones de valores y más. Este resumen es útil para que podamos evaluar la calidad de los datos, identificar patrones y tomar decisiones sobre la limpieza y el análisis de datos. Es una herramienta valiosa en nuestro proceso de Extracción, Transformación y Carga (ETL) de datos.

In [23]:
dfSummary(df_siniestros)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Id[object],1. 2016-00012. 2019-00513. 2019-00434. 2019-00445. 2019-00456. 2019-00467. 2019-00478. 2019-00489. 2019-004910. 2019-005011. other,1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)672 (98.5%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADXElEQVR4nO3cwWkbYRRG0TdGOPZGQhhcQtKDi0ixKUJFpAdjPIvY8WaySBYpINwB5ZwK/sWFh2A+Ldu2DRRu9n4A/w+xkREbGbGRERsZsZERGxmxkREbGbGRuVmW5cuyLHd7P4TrdzMzn2fm094P4fo5o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRG5mZmvs/Mz70fwvVb/IsRFWeUjNjIWFeRsa4i44ySERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZExeCFj8ELGGSUjNjIGL2QMXsg4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHFGyYiNjNjIWFeRsa4i44ySERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZGxriJjXUXGGSUjNjIGL2QMXsg4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZGxriJjXUXGGSUjNjLWVWSsq8g4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiMjNjLWVWSsq8g4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkXFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyy8zcz8zt3g/hqn1s2/Z2eHh4+Ho8Hs97v4brta7ry7Is3w7H4/H89PT0djqd3vd+FNfn9fX17nK5nJ+fn28PMzOn0+n98fHxx94P42rdz/iBQEhsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERuYw8/szkL0fwnX6u63Duq4vl8vlPH8+A4F/bV3Xl5n58Fk4hY9t296sq8j4gUBGbGTERkZsZMRGRmxkxEZGbGTERuYXwgdpOf758ysAAAAASUVORK5CYII="">",0(0.0%)
2,Num victimas[int64],Mean (sd) : 1.0 (0.2)min < med < max:1.0 < 1.0 < 3.0IQR (CV) : 0.0 (5.8),3 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABs0lEQVR4nO3XQWrCUBCH8ZkiqQ/khZCaY3gAD+FhPYRbTxLkZdGoVF53hVJaTbToX77fNplhFh8RPedswKN7ufcBwCUm515w92Bmxcj9x5xzP3IW+PJnqO4e6rpexRirMcu7rtu5+5pYca1zX9Qixlgtl8u+LMv9kMUppelms6nati3MjFBxlbM//WZmZVnum6Z5H7E/jJgBfuDPFCQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiRM7n3AM3L3YGbFyPFjzrm/5T3PgFBvzN1DXderGGM1Zr7rup27r4n1u4tCTSlNhy5OKU1Pp9PEzKK7Dz5MWJzNZvPFYvERQjgMGez7/nW73c7btn1z9+6/DnxQh5zz/reHnwblSFGwSZ17AAAAAElFTkSuQmCC"">",0(0.0%)
3,Fecha[datetime64[ns]],"Min: 2016-01-01Max: 2021-12-30Duration: 2,190 days",589 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABNCAYAAACxBha+AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACkUlEQVR4nO3dTYriQByG8X+NrUSFiAi68wIDs3DpIeawc4RZeAJv4E4QiZjpYIyZzcAsuv1ItXnNx/PblwT6KVNVaYnL89wAhW+vvgC0B7FB5q2MD3XO9c2s5zH0lOf5+7OvB9Xw9Nicc/3JZPIzDMNx0bGHw2HvnPulCu4Lk8KMiVFYGd9svTAMx8vl8n00GiWPDoqiKFitVuPdbtczs9L/iF+ZFGb6idEEpdxGzcxGo1EynU7/FBzWL+ViPuc1Kcz0E6MpSoutLjwnhZl2YjQCu1HIEBtkiA0yxAaZRmwQPM/Lwsvl0i3jevC52sfme16WpmmQZdn3JEl+m5nPbhQF1T428zwv22w24/V6PTifz50yLw7/NSE2Myt+Xrbf71txTlal59SVii3Lsq6Zhc65IsNYe11RtefUN2NTLrzjOO6mafpjPp93Op3Ow7dD1l43Veo59dXY1Avv0+n0FgTBYLFYJLPZbP/oONZe91XlOfWtb7aXLLyHwyFrr4a6u2Zj4Y1n4QkCZIgNMsQGmUqds9WJ55mgWYt/u0BsHnzPBM3a/dsFYvPgeybY9t8uENsXFD0T/Ke1R0NsECDDN5tYmzcWxCbU9o0FsQm1fWNBbC9Qh41FGbd7YsMHZd3uiQ0flHW7JzZc9ezbPedskCE2yBAbZIgNMsQGGWKDDLFBhtggQ2yQITbIEBtkiA0yxAYZYoMMsUGG2CBDbJAhNsgQG2SIDTLEBhl+XVUTTXghCbHVQFNeSEJsNdCUF5IQW43U/YUkbBAgQ2yQITbIEBtkiA0yxAYZYoMMsUGG2CBDbJAhNsgQG2SIDTLEBhlig8zd/2eLoigo8oHH4zEwM4vjONhut4OmjavTtarH3WvlL6rovNJurq/hAAAAAElFTkSuQmCC"">",0(0.0%)
4,Año[int64],Mean (sd) : 2018.2 (1.7)min < med < max:2016.0 < 2018.0 < 2021.0IQR (CV) : 3.0 (1201.6),6 distinct values,"<img src = ""data:image/png;ba

## Tratamiento y transformación de datos DF_VICTIMAS

In [24]:
df_victimas.head(5)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [25]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [26]:
df_victimas.describe()

,FECHA,AAAA,MM,DD
count,717,717.000000,717.000000,717.000000
mean,2018-09-09 01:36:24.100418304,2018.174338,6.688982,15.960948
min,2016-01-01 00:00:00,2016.000000,1.000000,1.000000
25%,2017-04-01 00:00:00,2017.000000,4.000000,9.000000
50%,2018-07-07 00:00:00,2018.000000,7.000000,16.000000
75%,2019-12-18 00:00:00,2019.000000,10.000000,23.000000
max,2021-12-30 00:00:00,2021.000000,12.000000,31.000000
std,NaN,1.672655,3.575140,8.683358


Los nombres de las columnas están en formato mixto, con algunas en mayúsculas y otras en minúsculas. 
Para seguir buenas prácticas, se normalizan a un formato consistente.

In [27]:
# Cambiar la primera letra a mayúscula y reemplazar guiones por espacios
df_victimas.columns = df_victimas.columns.str.capitalize().str.replace("_", " ")

# Renombrar columnas específicas
df_victimas.rename(columns={"Aaaa": "Año",
                            "Id hecho": "Id",
                              "Mm": "Mes",
                              "Dd": "Día"}, inplace=True)

In [28]:
df_victimas.columns

Index(['Id', 'Fecha', 'Año', 'Mes', 'Día', 'Rol', 'Victima', 'Sexo', 'Edad',
       'Fecha fallecimiento'],
      dtype='object')

In [29]:
num_filas2 = df_victimas.shape[0]
num_columnas2 = df_victimas.shape[1]
print(f'Número de filas: {num_filas2}')
print(f'Número de columnas: {num_columnas2}')

Número de filas: 717
Número de columnas: 10


In [30]:
df_victimas.isnull().sum()

Id                     0
Fecha                  0
Año                    0
Mes                    0
Día                    0
Rol                    0
Victima                0
Sexo                   0
Edad                   0
Fecha fallecimiento    0
dtype: int64

No se encuentran datos nulos 

Se procede a realizar la corrección del formato de las siguientes columnas:

- `Fecha Fallecimiento`: Actualmente, esta columna muestra la fecha junto con la hora, pero necesitamos que solo muestre la fecha en formato 'año-mes-día'.

- `Edad`: El formato actual de esta columna es 'object', sin embargo, necesitamos que esté en formato 'float' para poder aplicar métodos estadísticos.

In [31]:
# Convertir la columna "FECHA_FALLECIMIENTO" a datetime
df_victimas["Fecha fallecimiento"] = pd.to_datetime(df_victimas["Fecha fallecimiento"], format='%Y-%m-%d', errors='coerce')

# Filtrar las fechas no nulas (diferentes de NaT) y formatearlas para mostrar solo la fecha
df_victimas["Fecha fallecimiento"] = df_victimas["Fecha fallecimiento"].dt.strftime('%Y-%m-%d')

# Reemplazar las fechas en cero con espacios vacíos
df_victimas["Fecha fallecimiento"].replace("0", "", inplace=True)
df_victimas.head(5)


,Id,Fecha,Año,Mes,Día,Rol,Victima,Sexo,Edad,Fecha fallecimiento
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,NaN
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01


In [32]:
# Convierte la columna 'EDAD' a tipo numérico (float) y maneja los errores
df_victimas["Edad"] = pd.to_numeric(df_victimas["Edad"], errors="coerce")

# Reemplaza los valores no válidos (espacios, 'SD') con NaN
df_victimas["Edad"].replace([" ", "SD"], np.nan, inplace=True)

# Rellena los valores NaN con 0.0
df_victimas["Edad"].fillna(0.0, inplace=True)

# Convierte la columna 'EDAD' a tipo entero (int)
df_victimas["Edad"] = df_victimas["Edad"].astype(int)

# Verifica los valores únicos en la columna 'EDAD'
df_victimas["Edad"].unique()



array([19, 70, 30, 18, 29, 22, 16, 59, 65, 34, 41, 50, 38, 21, 52, 36, 20,
       54,  0, 56, 24, 78, 79, 26, 57, 37, 58, 23, 60, 42, 53, 51, 40, 87,
       76, 75, 35, 80, 43, 45, 67, 27, 55, 49, 81, 25, 33, 46, 83, 39, 28,
        7, 48,  4, 82, 32, 17, 47, 61, 10, 95, 73, 84, 66, 85,  1, 15, 13,
       77, 44, 31, 62, 74, 71, 11, 86, 69, 72, 12, 63, 92, 68, 91, 64,  5,
       88])

Se introduce la columna de `Rango de edad` como una derivación de la información contenida en la columna `Edad`, con el propósito de enriquecer y facilitar el análisis de los datos.

In [33]:
# Definir los límites de los rangos de edad y las etiquetas
bins = [0, 20, 40, 60, 80, 100]
labels = ['Menos de 20', 'Entre 21-40', 'Entre 41-60', 'Entre 61-80', 'Más de 80']

# Crear la columna de rango etario directamente en el DataFrame
df_victimas['Rango de edad'] = pd.cut(df_victimas['Edad'], bins=bins, labels=labels, right=False)

df_victimas.head(3)


,Id,Fecha,Año,Mes,Día,Rol,Victima,Sexo,Edad,Fecha fallecimiento,Rango de edad
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01,Menos de 20
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02,Entre 61-80
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03,Entre 21-40


 Se lleva a cabo un proceso de limpieza y optimización de los datos al identificar y eliminar columnas redundantes presentes en ambos conjuntos de datos (df_siniestros y df_victimas). En este caso, las columnas "Fecha," "Año," "Mes," "Día," y "Victima" en el conjunto de datos df_victimas contienen información duplicada que ya está presente en el conjunto de datos df_siniestros. Por lo tanto, estas columnas se eliminan del conjunto de datos df_victimas para evitar redundancias y simplificar el análisis posterior.

In [34]:
# Se eliminan las columnas repetidas
df_victimas = df_victimas.drop(['Fecha', 'Año', 'Mes', 'Día', 'Victima'], axis=1)
df_victimas.columns

Index(['Id', 'Rol', 'Sexo', 'Edad', 'Fecha fallecimiento', 'Rango de edad'], dtype='object')

El siguiente código `dfSummary(df_victimas)` se utiliza para resumir y comprender rápidamente el conjunto de datos en nuestro DataFrame `df_victimas`. Proporciona estadísticas descriptivas, información sobre valores faltantes, distribuciones de valores y más. Este resumen es útil para que podamos evaluar la calidad de los datos, identificar patrones y tomar decisiones sobre la limpieza y el análisis de datos. Es una herramienta valiosa en nuestro proceso de Extracción, Transformación y Carga (ETL) de datos.

In [35]:
dfSummary(df_victimas)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Id[object],1. 2017-00352. 2017-00503. 2016-00414. 2018-00155. 2020-00636. 2017-01267. 2019-00108. 2017-00269. 2017-011210. 2018-009811. other,3 (0.4%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)696 (97.1%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADm0lEQVR4nO3cQWojZxRG0VdGOO1JCSHwEpI9eBG92F6EFpE9GOMaxB1PlEECMenOJNC3QDlnZDT6BxeeDfq8XK/XgcLd3g/g/0NsZMRGRmxkxEZGbGTERkZsZMRGZpmZh5mZ6/X6tvNbuHGH8/n8eWZmWZYvguNHulvX9bSu62lm7vd+DLfN72xkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERuawbdvLXz+/7/oSbp4NApnFv8yi4nc2MnfLsvyyLMunvR/C7bubmZ9n5qe9H8Ltc0bJiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MjczcyvM/P73g/h9tkgkHFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiMjNjLWVWSsq8g4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkXFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkXFGyYiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyIiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlMwyMw8zc7/3Q7g579fr9e3jB4fz+fx5XdfTXi/iNm3b9rIsy5ePwR3WdT09PT29HY/Hr3s+jtvx+vr66XK5nJ6fn+9n5u/YZmaOx+PXx8fH33Z7Hbfo4Z8f+AOBjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNzGHmz6+E7P0Qbse/9XTYtu3lcrmc5jtfCYH/atu2l5l5//iZr4Xzo3zztXDrKjL+QCAjNjJiIyM2MmIjIzYyYiMjNjJiI/MHTYp+Vs//7zEAAAAASUVORK5CYII="">",0(0.0%)
2,Rol[object],1. CONDUCTOR2. PEATON3. PASAJERO_ACOMPAÑANTE4. CICLISTA5. SD,330 (46.0%)267 (37.2%)80 (11.2%)29 (4.0%)11 (1.5%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABzCAYAAACPdnBjAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACaklEQVR4nO3dQWrbUBiF0d/GpPVExgSyjCwgi+hiuwgvIPsIJhrUaSbuoKUUmriGPN/nSues4MV8SEjoksXxeCxIWPY+APMhNmLERsyiqtZVddP7IFfq9Xg8HnofYipWt7e3X4Zh2PY+yDUax3G/WCy+Cq6N1TAM24eHh8Nms3npfZhr8vz8/Hm3222fnp5uqkpsDayqqjabzcvd3d233oe5QuveB5gSDwjEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERsyq6ufnNL0Pcm38Ju2txnHc73a7bfmc5i/jOO6r6rX3OabCZ+Gn+Sy8oYXdKCkeEIiZ023ULbGz2ayrLKX6m8W6ylLqOsxpXeXVTmceEIgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI0ZsxIiNmFmsq6b+9/0vZrOuspTqz2fhxFhXEeMBgZhWt1G3KP6pybrKcolzfHhdZbnEuVqtqyb92oQ2PCAQIzZixEaM2IgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzEfXldZLnGuJusqyyXO4bNwYqyriDl1ZXO1oql3By9GLLT25uDFiIVLODV4MWKhKS91iREbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsRHz5rrKYopLeHddZTFFazYIxFhXEfPnlc2VjIv6va6ypuLSlsMwbO/v7+vXnG8O/3eUTpZVVev1+nvvgzB9XuoSIzZixEaM2IgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI2ZZVXU4HD71PgjTtxzHcf/4+GhNxcXZIBBjXUWMBwRixEbMDxqt/w3kjlieAAAAAElFTkSuQmCC"">",0(0.0%)
3,Sexo[object],1. MASCULINO2. FEMENINO3. SD,545 (76.0%)166 (23.2%)6 (0.8%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABFCAYAAABdVZTTAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABnklEQVR4nO3bQWrCUBSF4RMJUicvhICLcREutot4uwnBN6jiJB3UgnQoL+dd7P+t4IJ/1RRPt66rAIdd6wPwf3SSDpL2rQ9Bdfd1Xa+tj3jWT9N0TimNrQ9BXaWUpeu6z0jB9Sml8XQ6XYdhuLU+BnVcLpePnPM4z/NeUpzYJGkYhtvxePxqfQyqOrQ+4C8eEGBDbLAhNtgQG2yIDTbEBhtigw2xwYbYYENssCE22BAbbIgNNsQGm176+f1T60NQT9TXsy+lLDnnUQF//4TXlVIWSffWdzxjg/C+wm0QOqZ8cIn4zhbuLxJ1hFtXRVwFoY5Q66qoqyDUEXFdxVPxm+KfurAhNtgQG2yIDTbEBhtigw2xwYbYYENssCE22BAbbIgNNsQGG2KDTah1VZQ7sI1w66qIqyDUwQYBNqyrYMMDAmx+P0bFRxe21k/TdJYk5nPY2i6lND52o5EeEvCG+M4GG2KDDbHBhthgQ2ywITbYEBtsiA02xAYbYoMNscGG2GBDbLAhNtgQG2yIDTb9YzonMZ/DxtggwIYpH2y+ARDSg2w2upFrAAAAAElFTkSuQmCC"">",0(0.0%)
4,Edad[int32],Mean (sd) : 39.1 (22.0)min < med < max:0.0 < 35.0 < 95.0IQR (CV) : 31.0 (1.8),86 distinct values,"<img src = ""data:image/png;base64, iVBORw0

## DATASETS UNIFICADOS 

Los datasets transformados anteriormente se combinan mediante la función concat para unificar la información.
Asi como se presenta a continuación. 

In [36]:
# Combina los dos DataFrames por columnas y crea un nuevo DataFrame llamado 'siniestros'
siniestros_clean = pd.merge(df_victimas, df_siniestros, on='Id', how='left')

# Ahora 'siniestros_clean' contendrá todas las columnas de ambos DataFrames
siniestros_clean.columns

Index(['Id', 'Rol', 'Sexo', 'Edad', 'Fecha fallecimiento', 'Rango de edad',
       'Num victimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora int',
       'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Pos x', 'Pos y', 'Participantes',
       'Victima', 'Acusado', 'coordenada_x', 'coordenada_y',
       'Día de la Semana', 'Barrios'],
      dtype='object')

In [45]:
siniestros_clean.head(5)

,Id,Rol,Sexo,Edad,Fecha fallecimiento,Rango de edad,Num victimas,Fecha,Año,Mes,...,Comuna,Pos x,Pos y,Participantes,Victima,Acusado,coordenada_x,coordenada_y,Día de la Semana,Barrios
0,2016-0001,CONDUCTOR,MASCULINO,19,2016-01-01,Menos de 20,1,2016-01-01,2016,1,...,8.0,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378,Viernes,"Villa Soldati, Villa Riachuelo y Villa Lugano"
1,2016-0002,CONDUCTOR,MASCULINO,70,2016-01-02,Entre 61-80,1,2016-01-02,2016,1,...,9.0,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420,Sábado,"Liniers, Mataderos y Parque Avellaneda"
2,2016-0003,CONDUCTOR,MASCULINO,30,2016-01-03,Entre 21-40,1,2016-01-03,2016,1,...,1.0,-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878,Domingo,"Retiro, San Nicolás, Puerto Madero, San Telmo,..."
3,2016-0004,CONDUCTOR,MASCULINO,18,NaN,Menos de 20,1,2016-01-10,2016,1,...,8.0,-58.46503904,-34.68092974,MOTO-SD,MOTO,SD,99840.652248,94269.165344,Domingo,"Villa Soldati, Villa Riachuelo y Villa Lugano"
4,2016-0005,CONDUCTOR,MASCULINO,29,2016-02-01,Entre 21-40,1,2016-01-21,2016,1,...,1.0,-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS,106980.328279,100752.169158,Jueves,"Retiro, San Nicolás, Puerto Madero, San Telmo,..."


### EXPLORACIÓN DEL NUEVO DATAFRAME

In [37]:
siniestros_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     717 non-null    object        
 1   Rol                    717 non-null    object        
 2   Sexo                   717 non-null    object        
 3   Edad                   717 non-null    int32         
 4   Fecha fallecimiento    648 non-null    object        
 5   Rango de edad          717 non-null    category      
 6   Num victimas           702 non-null    float64       
 7   Fecha                  702 non-null    datetime64[ns]
 8   Año                    702 non-null    float64       
 9   Mes                    702 non-null    float64       
 10  Día                    702 non-null    float64       
 11  Hora int               701 non-null    float64       
 12  Lugar del hecho        702 non-null    object        
 13  Tipo 

Como se logra visualizar nos encontramos con datos cambiados de formato (Num Victimas, Año, Mes, Día) por ello procedemos a realizar la respectiva corrección.

In [38]:
siniestros_clean["Num victimas"] = siniestros_clean["Num victimas"].fillna(0).astype('Int64')
siniestros_clean["Num victimas"].unique()

<IntegerArray>
[1, 2, 0, 3]
Length: 4, dtype: Int64

In [39]:
siniestros_clean["Año"] = siniestros_clean["Año"].fillna(0).astype('Int64')
siniestros_clean["Año"].unique()

<IntegerArray>
[2016, 0, 2017, 2018, 2019, 2020, 2021]
Length: 7, dtype: Int64

In [40]:
siniestros_clean["Mes"] = siniestros_clean["Mes"].fillna(0).astype('Int64')
siniestros_clean["Mes"].unique()

<IntegerArray>
[1, 2, 3, 4, 0, 5, 6, 7, 8, 9, 10, 11, 12]
Length: 13, dtype: Int64

In [41]:
siniestros_clean["Día"] = siniestros_clean["Día"].fillna(0).astype('Int64')
siniestros_clean["Día"].unique()

<IntegerArray>
[ 1,  2,  3, 10, 21, 24, 29,  8, 14, 15, 17, 28,  4, 12, 13, 19, 23, 30, 31,
 11,  0, 20, 22, 25, 26, 16, 18, 27,  7,  9,  5,  6]
Length: 32, dtype: Int64

El siguiente código `dfSummary(siniestros_clean)` se utiliza para resumir y comprender rápidamente el conjunto de datos en nuestro DataFrame `siniestros_clean`. Proporciona estadísticas descriptivas, información sobre valores faltantes, distribuciones de valores y más. Este resumen es útil para que podamos evaluar la calidad de los datos, identificar patrones y tomar decisiones sobre la limpieza y el análisis de datos. Es una herramienta valiosa en nuestro proceso de Extracción, Transformación y Carga (ETL) de datos.

In [42]:
dfSummary(siniestros_clean)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Id[object],1. 2017-00352. 2017-00503. 2016-00414. 2018-00155. 2020-00636. 2017-01267. 2019-00108. 2017-00269. 2017-011210. 2018-009811. other,3 (0.4%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)2 (0.3%)696 (97.1%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADm0lEQVR4nO3cQWojZxRG0VdGOO1JCSHwEpI9eBG92F6EFpE9GOMaxB1PlEECMenOJNC3QDlnZDT6BxeeDfq8XK/XgcLd3g/g/0NsZMRGRmxkxEZGbGTERkZsZMRGZpmZh5mZ6/X6tvNbuHGH8/n8eWZmWZYvguNHulvX9bSu62lm7vd+DLfN72xkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERuawbdvLXz+/7/oSbp4NApnFv8yi4nc2MnfLsvyyLMunvR/C7bubmZ9n5qe9H8Ltc0bJiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MjczcyvM/P73g/h9tkgkHFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiMjNjLWVWSsq8g4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkXFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkXFGyYiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlIzYyIiNjHUVGesqMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVknFEyYiNj8ELG4IWMM0pGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxlnlMwyMw8zc7/3Q7g579fr9e3jB4fz+fx5XdfTXi/iNm3b9rIsy5ePwR3WdT09PT29HY/Hr3s+jtvx+vr66XK5nJ6fn+9n5u/YZmaOx+PXx8fH33Z7Hbfo4Z8f+AOBjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNzGHmz6+E7P0Qbse/9XTYtu3lcrmc5jtfCYH/atu2l5l5//iZr4Xzo3zztXDrKjL+QCAjNjJiIyM2MmIjIzYyYiMjNjJiI/MHTYp+Vs//7zEAAAAASUVORK5CYII="">",0(0.0%)
2,Rol[object],1. CONDUCTOR2. PEATON3. PASAJERO_ACOMPAÑANTE4. CICLISTA5. SD,330 (46.0%)267 (37.2%)80 (11.2%)29 (4.0%)11 (1.5%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABzCAYAAACPdnBjAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACaklEQVR4nO3dQWrbUBiF0d/GpPVExgSyjCwgi+hiuwgvIPsIJhrUaSbuoKUUmriGPN/nSues4MV8SEjoksXxeCxIWPY+APMhNmLERsyiqtZVddP7IFfq9Xg8HnofYipWt7e3X4Zh2PY+yDUax3G/WCy+Cq6N1TAM24eHh8Nms3npfZhr8vz8/Hm3222fnp5uqkpsDayqqjabzcvd3d233oe5QuveB5gSDwjEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsREjNmLERsyq6ufnNL0Pcm38Ju2txnHc73a7bfmc5i/jOO6r6rX3OabCZ+Gn+Sy8oYXdKCkeEIiZ023ULbGz2ayrLKX6m8W6ylLqOsxpXeXVTmceEIgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI0ZsxIiNmFmsq6b+9/0vZrOuspTqz2fhxFhXEeMBgZhWt1G3KP6pybrKcolzfHhdZbnEuVqtqyb92oQ2PCAQIzZixEaM2IgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzEfXldZLnGuJusqyyXO4bNwYqyriDl1ZXO1oql3By9GLLT25uDFiIVLODV4MWKhKS91iREbMWIjRmzEiI0YsREjNmLERozYiBEbMWIjRmzEiI0YsRHz5rrKYopLeHddZTFFazYIxFhXEfPnlc2VjIv6va6ypuLSlsMwbO/v7+vXnG8O/3eUTpZVVev1+nvvgzB9XuoSIzZixEaM2IgRGzFiI0ZsxIiNGLERIzZixEaM2IgRGzFiI2ZZVXU4HD71PgjTtxzHcf/4+GhNxcXZIBBjXUWMBwRixEbMDxqt/w3kjlieAAAAAElFTkSuQmCC"">",0(0.0%)
3,Sexo[object],1. MASCULINO2. FEMENINO3. SD,545 (76.0%)166 (23.2%)6 (0.8%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAABFCAYAAABdVZTTAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMSwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/bCgiHAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABnklEQVR4nO3bQWrCUBSF4RMJUicvhICLcREutot4uwnBN6jiJB3UgnQoL+dd7P+t4IJ/1RRPt66rAIdd6wPwf3SSDpL2rQ9Bdfd1Xa+tj3jWT9N0TimNrQ9BXaWUpeu6z0jB9Sml8XQ6XYdhuLU+BnVcLpePnPM4z/NeUpzYJGkYhtvxePxqfQyqOrQ+4C8eEGBDbLAhNtgQG2yIDTbEBhtigw2xwYbYYENssCE22BAbbIgNNsQGm176+f1T60NQT9TXsy+lLDnnUQF//4TXlVIWSffWdzxjg/C+wm0QOqZ8cIn4zhbuLxJ1hFtXRVwFoY5Q66qoqyDUEXFdxVPxm+KfurAhNtgQG2yIDTbEBhtigw2xwYbYYENssCE22BAbbIgNNsQGG2KDTah1VZQ7sI1w66qIqyDUwQYBNqyrYMMDAmx+P0bFRxe21k/TdJYk5nPY2i6lND52o5EeEvCG+M4GG2KDDbHBhthgQ2ywITbYEBtsiA02xAYbYoMNscGG2GBDbLAhNtgQG2yIDTb9YzonMZ/DxtggwIYpH2y+ARDSg2w2upFrAAAAAElFTkSuQmCC"">",0(0.0%)
4,Edad[int32],Mean (sd) : 39.1 (22.0)min < med < max:0.0 < 35.0 < 95.0IQR (CV) : 31.0 (1.8),86 distinct values,"<img src = ""data:image/png;base64, iVBORw0

### SE GUARDA ARCHIVO COMO CSV PARA FACILITAR SU PROCESAMIENTO

In [43]:
siniestros_clean.to_csv('DATA/siniestros_clean.csv', index=False)